# Objetivo:

Entregar a LLM as imagens para serem descritas, com o contexto do chunk onde se encontra

## Passo a Passo:

## 1 - Separar todos os chunks do texto
## 2 - Encontrar os chunks que contem imagens

In [10]:
import os
import sys

import regex as re

# Pega o diretório atual do notebook
notebook_dir = os.getcwd() # ou os.path.dirname(__file__) se fosse um script .py

# Assume que 'src' está no mesmo nível do notebook ou um nível acima
# Ajuste '..' conforme a estrutura do seu projeto
project_root = os.path.abspath(os.path.join(notebook_dir, '..')) # Volta um diretório

# Se o 'src' estiver diretamente no mesmo nível do notebook:
# project_root = notebook_dir

# Adiciona o diretório raiz do projeto ao sys.path
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [11]:
from PIL import Image
import io
import base64
from src.classe_gemini import GeminiApiClient
from src.retrieve_img import retrieve_image_bytes_from_db
from src.interact_database_sql import save_image_description_to_db, get_image_description_from_db, non_described_images

import sqlite3

In [12]:
# --- Busca dos bytes da imagem no DB ---
image_name = "_page_0_Picture_1.jpeg"
image_bytes = retrieve_image_bytes_from_db(filename=image_name)

if image_bytes:
    # --- Codifica os bytes da imagem para Base64 ---
    # A classe GeminiApiClient espera a imagem em formato base64
    base64_image = base64.b64encode(image_bytes).decode('utf-8')

    # --- Prepara os dados para a API Gemini (usando a estrutura esperada pela classe) ---
    image_part = {
        "inlineData": {
            "mimeType": "image/jpeg", # Confirme que este é o tipo MIME correto da sua imagem
            "data": base64_image
        }
    }

    # --- Inicializa o cliente Gemini API ---
    # Certifique-se de que a variável de ambiente 'GOOGLE_API_KEY' está definida com sua chave de API
    try:
        api_key = os.environ.get("GOOGLE_API_KEY")
        if not api_key:
            raise ValueError("A variável de ambiente 'GOOGLE_API_KEY' não está definida.")
        
        gemini_client = GeminiApiClient(api_key=api_key)
    except ValueError as e:
        print(f"Erro de configuração da API: {e}")
        exit() # Encerra o programa se a chave da API não estiver configurada


### 1 - Separar todos os chunks do texto

In [13]:
# Regex para encontrar as linhas que começam com o padrão de data

diary_text_path = r"C:\Users\fuedj\Documents\Code\RAG_Dr_Voss_v2\drvossv2\data\dr_voss_md.md"

with open(diary_text_path, 'r', encoding='utf-8') as md:
    diary_text = md.read()

In [14]:
date_pattern = r"(\d{1,2})(?:st|nd|rd|th)? Day of ([A-Za-z]+) (18\d{2}) - ([A-Za-z\s]+)"

matches = [(match, match.start()) for match in re.finditer(date_pattern, diary_text)]

all_chunks = []
for i, (match, start_pos) in enumerate(matches):
    # Extract date components and title
    day, month, year, title = match.groups()
    # Determine the end position (next match's start or end of text)
    end_pos = matches[i + 1][1] if i + 1 < len(matches) else len(diary_text)
    # Get the content including the date line
    content = diary_text[start_pos:end_pos].strip()
    all_chunks.append({
        'day': day,
        'month': month,
        'year': year,
        'title': title.strip(),
        'content': content
    })

In [15]:
len(all_chunks)

123

In [16]:
all_chunks[:3]

[{'day': '1',
  'month': 'Frostfall',
  'year': '1855',
  'title': 'Arrival in the Capital of Veridia',
  'content': "1st Day of Frostfall 1855 - Arrival in the Capital of Veridia\n\n![](_page_0_Picture_1.jpeg)\n\nToday marks my arrival in the capital city of Veridia, a place teeming with vibrant cultural heritage and an unyielding commitment to progress. Under Queen Isolde's famed patronage, the arts flourished here. Walking through the grand avenues, I was captivated by the array of sculptures and paintings, bearing testament to her legacy. My first stop was the regal Assembly House, where the Assembly of Voices, Veridia's main legislative body, convenes. The debate inside, I was told, revolved around initiatives to fulfill Veridia's ambitious goal of using 80% renewable energy by 2050. This pervasive focus on sustainability is palpable, infusing the city's very lifeblood.\n\nAs evening crept in, I joined locals indulging in Zelphar stew, a traditional Veridian dish that warms the so

### 2 - Encontrar os chunks que contém imagens

In [17]:
# Filter chunks that contain the image 'jpeg' in their content
image_chunks = [
    chunk for chunk in all_chunks
    if ".jpeg)" in chunk['content']
]

In [18]:
image_chunks[:3]

[{'day': '1',
  'month': 'Frostfall',
  'year': '1855',
  'title': 'Arrival in the Capital of Veridia',
  'content': "1st Day of Frostfall 1855 - Arrival in the Capital of Veridia\n\n![](_page_0_Picture_1.jpeg)\n\nToday marks my arrival in the capital city of Veridia, a place teeming with vibrant cultural heritage and an unyielding commitment to progress. Under Queen Isolde's famed patronage, the arts flourished here. Walking through the grand avenues, I was captivated by the array of sculptures and paintings, bearing testament to her legacy. My first stop was the regal Assembly House, where the Assembly of Voices, Veridia's main legislative body, convenes. The debate inside, I was told, revolved around initiatives to fulfill Veridia's ambitious goal of using 80% renewable energy by 2050. This pervasive focus on sustainability is palpable, infusing the city's very lifeblood.\n\nAs evening crept in, I joined locals indulging in Zelphar stew, a traditional Veridian dish that warms the so

In [23]:
# --- Chamada da API ---
model_name = 'gemini-1.5-pro' # Ou 'gemini-1.5-pro' se preferir um modelo mais potente

chunk_description = "1st Day of Frostfall 1855 - Arrival in the Capital of Veridia\n\n!\n\nToday marks my arrival in the capital city of Veridia, a place teeming with vibrant cultural heritage and an unyielding commitment to progress. Under Queen Isolde's famed patronage, the arts flourished here. Walking through the grand avenues, I was captivated by the array of sculptures and paintings, bearing testament to her legacy. My first stop was the regal Assembly House, where the Assembly of Voices, Veridia's main legislative body, convenes. The debate inside, I was told, revolved around initiatives to fulfill Veridia's ambitious goal of using 80% renewable energy by 2050. This pervasive focus on sustainability is palpable, infusing the city's very lifeblood.\n\nAs evening crept in, I joined locals indulging in Zelphar stew, a traditional Veridian dish that warms the soul as much as it pleases the palate. Each spoonful was a blend of flavors, rich and comforting after my long journey.\n\n####"

prompt = f"""
    
    You are describing a image from a fantasy world in a travel log journey of Doctor Voss, a woman visiting the capital of Veridia.
    Here is the context of the image: {chunk_description}.
    
    Describe **only** the image. Bring **only the description**. Be concise - 300 characters max.
    
    """
    
prompt_parts = [
    {"text": f"{prompt}"},
    image_part
]

print(f"Enviando imagem do banco de dados para a API Gemini (modelo: {model_name})...")

# Chama o método da classe GeminiApiClient
response_data = gemini_client.generate_multimodal_content(model_name, prompt_parts)

print("\n--- Resposta Bruta da API Gemini ---")
print(response_data) # Para ver a estrutura completa da resposta
print("-----------------------------------")

# Extrai o texto da resposta usando o método da classe
generated_text = gemini_client.extract_text_from_response(response_data)

if generated_text:
    print("\n--- Texto Descritivo do Gemini ---")
    print(generated_text)
    print("---------------------------------")
else:
    print("\nNão foi possível extrair texto da resposta do Gemini.")



Enviando imagem do banco de dados para a API Gemini (modelo: gemini-1.5-pro)...

--- Resposta Bruta da API Gemini ---
{'candidates': [{'content': {'parts': [{'text': 'A bustling street market stretches before the grand Assembly House, its pastel facade and colonnaded balconies overlooking the scene. Colorful awnings shade vendors and shoppers, carts laden with produce lining the sunlit avenue.\n'}], 'role': 'model'}, 'finishReason': 'STOP', 'avgLogprobs': -0.35646168778582316}], 'usageMetadata': {'promptTokenCount': 540, 'candidatesTokenCount': 41, 'totalTokenCount': 581, 'promptTokensDetails': [{'modality': 'IMAGE', 'tokenCount': 258}, {'modality': 'TEXT', 'tokenCount': 282}], 'candidatesTokensDetails': [{'modality': 'TEXT', 'tokenCount': 41}]}, 'modelVersion': 'gemini-1.5-pro-002', 'responseId': 'J1dUaIvZLq6k7dcPjazPkQ8'}
-----------------------------------

--- Texto Descritivo do Gemini ---
A bustling street market stretches before the grand Assembly House, its pastel facade and co

In [24]:
o = save_image_description_to_db(filename=image_name, description=generated_text)

Coluna 'descricao_gemini' já existe na tabela 'imagens' em C:\Users\fuedj\Documents\Code\RAG_Dr_Voss_v2\drvossv2\data\imagens.db.
Sucesso: Descrição da imagem para '_page_0_Picture_1.jpeg' salva no DB.


In [25]:
# Exemplo 1: Consultar por nome de arquivo
filename_to_query = image_name
description_by_filename = get_image_description_from_db(filename=filename_to_query)

if description_by_filename:
    print(f"\n--- Descrição recuperada por filename ('{filename_to_query}') ---")
    print(description_by_filename)
    print("-------------------------------------------------")
else:
    print(f"\nNenhuma descrição encontrada para '{filename_to_query}' por nome de arquivo.")


--- Descrição recuperada por filename ('_page_0_Picture_1.jpeg') ---
A bustling street market stretches before the grand Assembly House, its pastel facade and colonnaded balconies overlooking the scene. Colorful awnings shade vendors and shoppers, carts laden with produce lining the sunlit avenue.

-------------------------------------------------
